In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/roberta-base-mp-svr-5-folds-training/svr_head_3.pkl
/kaggle/input/roberta-base-mp-svr-5-folds-training/roberta_base_1.pt
/kaggle/input/roberta-base-mp-svr-5-folds-training/__results__.html
/kaggle/input/roberta-base-mp-svr-5-folds-training/roberta_base_3.pt
/kaggle/input/roberta-base-mp-svr-5-folds-training/roberta_base_2.pt
/kaggle/input/roberta-base-mp-svr-5-folds-training/roberta_base_4.pt
/kaggle/input/roberta-base-mp-svr-5-folds-training/svr_head_2.pkl
/kaggle/input/roberta-base-mp-svr-5-folds-training/svr_head_0.pkl
/kaggle/input/roberta-base-mp-svr-5-folds-training/__notebook__.ipynb
/kaggle/input/roberta-base-mp-svr-5-folds-training/roberta_base_0.pt
/kaggle/input/roberta-base-mp-svr-5-folds-training/__output__.json
/kaggle/input/roberta-base-mp-svr-5-folds-training/svr_head_4.pkl
/kaggle/input/roberta-base-mp-svr-5-folds-training/svr_head_1.pkl
/kaggle/input/roberta-base-mp-svr-5-folds-training/custom.css
/kaggle/input/roberta-base-svr-5-folds-training/svr_head_3

In [2]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel, AlbertModel, AlbertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
import tqdm
import gc
from sklearn.svm import SVR
import xgboost as xgb
import pickle

In [3]:
df_test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
df_test

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...
5,12537fe78,NaN,NaN,"To explain transitivity, let us look first at ..."
6,965e592c0,https://www.africanstorybook.org/#,CC BY 4.0,Milka and John are playing in the garden. Her ...


In [4]:
data_test = df_test['excerpt'].values
ind_test = df_test['id'].values
data_test[0], ind_test[0]

('My hope lay in Jack\'s promise that he would keep a bright light burning in the upper story to guide me on my course. On a clear night this light was visible from the village, but somehow or other I failed to take into account the state of the weather. The air was full of eddying flakes, which would render the headlight of a locomotive invisible a hundred yards distant. Strange that this important fact never occurred to me until I was fully a fourth of a mile from the village. Then, after looking in vain for the beacon light, the danger of my situation struck me, and I halted.\n"I am certain to go wrong," I said to myself.\n"It is out of my power to follow a direct course without something to serve as a compass. I will go back to the village and wait till morning."',
 'c0f722661')

RoBerta base

In [5]:
%%bash
cp ../input/roberta-base-save/rob.zip .
cp ../input/roberta-base-save/rob_tok.zip .
unzip rob.zip
unzip rob_tok.zip 
rm -r rob.zip rob_tok.zip 

Archive:  rob.zip
   creating: rob/
  inflating: rob/pytorch_model.bin   
  inflating: rob/config.json         
Archive:  rob_tok.zip
   creating: rob_tok/
  inflating: rob_tok/vocab.json      
  inflating: rob_tok/special_tokens_map.json  
  inflating: rob_tok/tokenizer_config.json  
  inflating: rob_tok/merges.txt      


In [6]:
tokenizer = RobertaTokenizer.from_pretrained(
    'rob_tok'
)

In [7]:
batch_size = 32

encoded_data_test = tokenizer.batch_encode_plus(
    data_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
ids_test_tensor = torch.tensor(df_test.index, dtype=torch.float)

dataset_test = TensorDataset(input_ids_test,
                            attention_masks_test,
                            ids_test_tensor)

dataloader_test = DataLoader(
    dataset_test,
    batch_size=batch_size
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
class BERTRegressor(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = RobertaModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(768, 1)
        self.dropout = torch.nn.Dropout(0.15)
        
    def forward(self, input_ids, attention_mask): #x - tokenized batch
        hidden = self.bert(input_ids, 
                           attention_mask=attention_mask)[0][:, 0, :]#CLS token output                                                          
        output = self.linear(self.dropout(hidden))
        return output

    
def get_bert_embeddings(embedder, dataloader, device = device):
    embedder.eval()
    embeddings_all = []
    for batch in tqdm.notebook.tqdm(dataloader):        
        batch = tuple(b.to(device) for b in batch)        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                 }
        with torch.no_grad():        
            output = embedder(**inputs)[0][:, 0, :]#CLS token output 

        embeddings_batch = output.detach().cpu().numpy()
        embeddings_all.append(embeddings_batch)

    return np.vstack(embeddings_all)

class BERTRegressorMP(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = RobertaModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(768, 1)
        self.dropout = torch.nn.Dropout(0.1)        
        
    def forward(self, input_ids, attention_mask):           
        last_hidden_state = self.bert(input_ids, 
                           attention_mask=attention_mask)[0]   
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        output = self.linear(self.dropout(mean_embeddings))
        return output
    
class EmbedderMP(torch.nn.Module):
    def __init__(self, model): 
        super().__init__()
        self.bert = model.bert
    
    def forward(self, input_ids, attention_mask): 
        last_hidden_state = self.bert(input_ids, 
                           attention_mask=attention_mask)[0]   
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    
def get_bert_embeddings_MP(embedder, dataloader, device = device):
    embedder.eval()
    embeddings_all = []
    for batch in tqdm.notebook.tqdm(dataloader):        
        batch = tuple(b.to(device) for b in batch)        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                 }
        with torch.no_grad():        
            output = embedder(**inputs)

        embeddings_batch = output.detach().cpu().numpy()
        embeddings_all.append(embeddings_batch)

    return np.vstack(embeddings_all)

In [10]:
import gc
gc.collect()

24

In [11]:
preds_all = []

roberta-base-svr-5-folds-training

In [12]:
model_ids = [0, 1, 2, 3, 4]

In [13]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressor()
    PATH = f'../input/roberta-base-svr-5-folds-training/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = model.bert.to(device)
    embeddings_test = get_bert_embeddings(embedder, dataloader_test)
    filename = f'../input/roberta-base-svr-5-folds-training/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

5

roberta-base-xgb-5-folds-training

In [14]:
model_ids = [0, 1, 2, 3, 4]

In [15]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressor()
    PATH = f'../input/roberta-base-xgb-5-folds-training/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = model.bert.to(device)
    embeddings_test = get_bert_embeddings(embedder, dataloader_test)
    filename = f'../input/roberta-base-xgb-5-folds-training/xgb_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

10

roberta-base-svr-5-folds-training-skf

In [16]:
model_ids = [0, 1, 2, 3, 4]

In [17]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressor()
    PATH = f'../input/roberta-base-svr-5-folds-training-skf/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = model.bert.to(device)
    embeddings_test = get_bert_embeddings(embedder, dataloader_test)
    filename = f'../input/roberta-base-svr-5-folds-training-skf/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

15

roberta-base-mp-svr-5-folds-training

In [18]:
model_ids = [0, 1, 2, 3, 4]

In [19]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressorMP()
    PATH = f'../input/roberta-base-mp-svr-5-folds-training/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = EmbedderMP(model).to(device)
    embeddings_test = get_bert_embeddings_MP(embedder, dataloader_test)
    filename = f'../input/roberta-base-mp-svr-5-folds-training/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

20

roberta-base-mp-xgb-5-folds-training

In [20]:
model_ids = [0, 1, 2, 3, 4]

In [21]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressorMP()
    PATH = f'../input/roberta-base-mp-xgb-5-folds-training/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = EmbedderMP(model).to(device)
    embeddings_test = get_bert_embeddings_MP(embedder, dataloader_test)
    filename = f'../input/roberta-base-mp-xgb-5-folds-training/xgb_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

25

roberta-base-mp-svr-5-folds-skf-training

In [22]:
model_ids = [0, 1, 2, 3, 4]

In [23]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressorMP()
    PATH = f'../input/roberta-base-mp-svr-5-folds-skf-training/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = EmbedderMP(model).to(device)
    embeddings_test = get_bert_embeddings_MP(embedder, dataloader_test)
    filename = f'../input/roberta-base-mp-svr-5-folds-skf-training/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

30

roberta-base-mp-xgb-5-folds-training-skf

In [24]:
model_ids = [0, 1, 2, 3, 4]

In [25]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressorMP()
    PATH = f'../input/roberta-base-mp-xgb-5-folds-training-skf/roberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = EmbedderMP(model).to(device)
    embeddings_test = get_bert_embeddings_MP(embedder, dataloader_test)
    filename = f'../input/roberta-base-mp-xgb-5-folds-training-skf/xgb_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

35

In [26]:
!rm -r rob rob_tok

Distill RoBerta

In [27]:
%%bash
cp ../input/distil-roberta-base-save/rob.zip .
cp ../input/distil-roberta-base-save/rob_tok.zip .
unzip rob.zip
unzip rob_tok.zip 
rm -r rob.zip rob_tok.zip 

Archive:  rob.zip
   creating: rob/
  inflating: rob/pytorch_model.bin   
  inflating: rob/config.json         
Archive:  rob_tok.zip
   creating: rob_tok/
  inflating: rob_tok/special_tokens_map.json  
  inflating: rob_tok/tokenizer_config.json  
  inflating: rob_tok/vocab.json      
  inflating: rob_tok/merges.txt      


In [28]:
tokenizer = RobertaTokenizer.from_pretrained(
    'rob_tok'
)

In [29]:
batch_size = 32

encoded_data_test = tokenizer.batch_encode_plus(
    data_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
ids_test_tensor = torch.tensor(df_test.index, dtype=torch.float)

dataset_test = TensorDataset(input_ids_test,
                            attention_masks_test,
                            ids_test_tensor)

dataloader_test = DataLoader(
    dataset_test,
    batch_size=batch_size
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


distil-roberta-base-svr-5-folds-training

In [30]:
model_ids = [0, 1, 2, 3, 4]

In [31]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressor()
    PATH = f'../input/distil-roberta-base-svr-5-folds-training/distroberta_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = model.bert.to(device)
    embeddings_test = get_bert_embeddings(embedder, dataloader_test)
    filename = f'../input/distil-roberta-base-svr-5-folds-training/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

40

In [32]:
!rm -r rob rob_tok

Albert base

In [33]:
%%bash
cp ../input/albert-base-save/al.zip .
cp ../input/albert-base-save/al_tok.zip .
unzip al.zip
unzip al_tok.zip 
rm -r al.zip al_tok.zip 

Archive:  al.zip
   creating: al/
  inflating: al/config.json          
  inflating: al/pytorch_model.bin    
Archive:  al_tok.zip
   creating: al_tok/
  inflating: al_tok/special_tokens_map.json  
  inflating: al_tok/tokenizer_config.json  
  inflating: al_tok/spiece.model     


In [34]:
tokenizer = AlbertTokenizer.from_pretrained(
    'al_tok'
)

In [35]:
batch_size = 32

encoded_data_test = tokenizer.batch_encode_plus(
    data_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
ids_test_tensor = torch.tensor(df_test.index, dtype=torch.float)

dataset_test = TensorDataset(input_ids_test,
                            attention_masks_test,
                            ids_test_tensor)

dataloader_test = DataLoader(
    dataset_test,
    batch_size=batch_size
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


albert-base-svr-5-folds-training

In [36]:
class BERTRegressor(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = AlbertModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(768, 1)
        self.dropout = torch.nn.Dropout(0.15)
        
    def forward(self, input_ids, attention_mask): #x - tokenized batch
        hidden = self.bert(input_ids, 
                           attention_mask=attention_mask)[0][:, 0, :]#CLS token output                                                          
        output = self.linear(self.dropout(hidden))
        return output

In [37]:
model_ids = [0, 1, 2, 3, 4]

In [38]:
for k in tqdm.notebook.tqdm(model_ids):
    model = BERTRegressor(pretrained_src = 'al')
    PATH = f'../input/albert-base-svr-5-folds-training/albert_base_{k}.pt'
    model.load_state_dict(torch.load(PATH), strict=False)
    embedder = model.bert.to(device)
    embeddings_test = get_bert_embeddings(embedder, dataloader_test)
    filename = f'../input/albert-base-svr-5-folds-training/svr_head_{k}.pkl'
    new_head = pickle.load(open(filename, 'rb'))
    preds = new_head.predict(embeddings_test)
    preds_all.append(preds.reshape((-1, 1)))
len(preds_all)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

45

In [39]:
!rm -r al al_tok

Final predictions

In [40]:
predictions_mean = np.hstack(preds_all).mean(axis = 1)
predictions_mean

array([-0.35438733, -0.50541347, -0.44422807, -2.41990233, -1.81491643,
       -1.24938991,  0.33677109])

In [41]:
ids = pd.DataFrame(ind_test, columns=['id'])['id']

In [42]:
sub_df = pd.DataFrame(predictions_mean, index=ids,
                      columns = ['target'])
sub_df

,target
id,
c0f722661,-0.354387
f0953f0a5,-0.505413
0df072751,-0.444228
04caf4e0c,-2.419902
0e63f8bea,-1.814916
12537fe78,-1.249390
965e592c0,0.336771


In [43]:
sub_df.to_csv('submission.csv')